This script uses the Census API to download and organize key Census/ACS variables of interest for specified geographic areas (census tracts in this example), and generates outputs for further summary.

In [450]:
import pandas as pd, numpy as np, census, us, csv, os, settings, requests as r, math
from math import sqrt
from census import Census
from us import states

In [451]:
c = Census(settings.CENSUS_KEY)

## Helper Functions

In [452]:
# pull dataframe for ACS 5-year estimates, set values as float

def acs5(vardict,state,county,yr):
    df = pd.DataFrame(c.acs5.get((vardict.keys()), geo={'for': 'tract:*', 'in': 'state:'+state+' county:'+county}, year=yr))
    df = df.rename(columns=vardict)
    clist = list(df.columns.values)
    for i in clist:
        if i != 'NAME' and i != 'state' and i != 'county' and i != 'tract':
            df[i] = df[i].astype(float)
    return df

In [453]:
# pull dataframe for SF1, set values as float

def sf1(vardict,state,county,yr):
    df = pd.DataFrame(c.sf1.get((vardict.keys()), geo={'for': 'tract:*', 'in': 'state:'+state+' county:'+county}, year=yr))
    df = df.rename(columns=vardict)
    clist = list(df.columns.values)
    for i in clist:
        if i != 'NAME' and i != 'state' and i != 'county' and i != 'tract':
            df[i] = df[i].astype(float)
    return df

In [454]:
# pull dataframe for SF3, set values as float

def sf3(vardict,state,county,yr):
    df = pd.DataFrame(c.sf3.get((vardict.keys()), geo={'for': 'tract:*', 'in': 'state:'+state+' county:'+county}, year=yr))
    df = df.rename(columns=vardict)
    clist = list(df.columns.values)
    for i in clist:
        if i != 'NAME' and i != 'state' and i != 'county' and i != 'tract':
            df[i] = df[i].astype(float)
    return df

In [455]:
# filter to specific list of tracts

def tractfilter(df,tractlist):
    df = df[df['tract'].isin(tractlist)]
    df = df.set_index(df.tract)
    return df

In [456]:
# pull lists of census tracts in study area in 2000 and 2010

def tractlist(csv):
    df = pd.read_csv(csv, dtype=str)
    df = df['tracts'].tolist()
    map(str,df)
    return df

In [457]:
# Collapse categories (counts only), using external key of old and new category names
# Key should be csv file previously read into dataframe

def collapse(df,keys):
    
    ix = df.index
    c = df.columns
    a = keys[keys['old'].isin(c)]
    col = a.new.unique()
    
    #create empty frame for output
    output = pd.DataFrame(index = ix, columns = col)
    
    #cycle through each new column to be created
    for u in col:
        
        #copy old structure and set whole column to 0
        for i in output.iterrows():
            output[u] = 0
        
        #cycle through key and add columns in old dataframe together according to key
        for i in keys.index.values:
            if (keys.new[i] == u) & (keys.old[i] in df.columns.values):
                output[u] = output[u] + df[keys.old[i]]
                
    return output

In [458]:
# Collapse categories (MOEs only), using external key of old and new category names
# Key should be csv file previously read into dataframe

def collapseMOE(df,keys):
    
    ix = df.index
    c = df.columns
    a = keys[keys['old'].isin(c)]
    col = a.new.unique()
    
    #create empty frame for output
    output = pd.DataFrame(index = ix, columns = col)
    
    #cycle through each new column to be created
    for u in col:
        r = keys['old'][keys.new == u].tolist()
        d = df[r].astype('float')
        e = d**2
        f = e.sum(axis=1)
        output[u] = f.apply(sqrt)
    
    return output

In [459]:
# Aggregate census tracts for tables with counts

# Need to figure out a way to sum the row if contains "(Estimate)" and do the margin of error aggregation if contains "(MOE)"
# MOE aggregation formula is square root of sum of squared MOEs

def agg(df):
    df.loc['Aggregate'] = 0
    for i in df.columns:
        if "(Estimate)" in i:
            df[i]['Aggregate'] = sum(df[i])
        elif "(MOE)" in i:
            df[i]['Aggregate'] = sqrt(sum(df[i]**2))
        elif df[i].dtypes == 'float64':
            df[i]['Aggregate'] = sum(df[i])
    new = df.loc['Aggregate']
    df.drop('Aggregate')
    return new

In [460]:
# read csv variable dictionary

variablesheet = pd.read_csv('C:\Users\pksohn\Box Sync\Berkeley Coursework\CP201A Planning Methods Gateway Fall\Assignments\Neighborhood Profile\Variables\Master Variable Dictionary.csv', dtype='str')

In [461]:
# Pull variable dictionary for specific pull using previously defined external dictionary

def vardict(pull, year, survey):
    v = variablesheet.set_index('var')
    return v['label'][(v.pull == pull) & (v.year == year) & (v.survey == survey)].to_dict()

In [462]:
# Drew's function to concatenate dataframes 

def concatenate_dataframes(*dfs):
    if len(dfs) < 2:
        return dfs[0]
    
    masterframe = dfs[0]
    for df in dfs[1:]:
        masterframe = pd.concat([masterframe, df], axis = 1)
    masterframe.T.groupby
    return masterframe

## Geographic Parameters

In [463]:
# Set FIPS codes for area

CA = '06'
alameda = '001'

In [464]:
# get csv files of census tracts in study area

tracts10 = tractlist('2010_tracts.csv')
tracts00 = tractlist('2000_tracts.csv')

## 2008-2012 5-year ACS descriptives

##### Basic counts

In [465]:
var_basic10 = vardict('basic','2010','ACS5')

In [466]:
#pull alameda county basic descriptives
#filter for study area
#check counts

alameda_basic10 = acs5(var_basic10,CA,alameda,2012)
basic10 = tractfilter(alameda_basic10,tracts10)
len(basic10)

11

In [467]:
#look at basic data

basic10

,Population (Estimate),Population (MOE),Total Households (Estimate),Total Households (MOE),county,state,tract
tract,,,,,,,
401000,4997,475,2288,140,001,06,401000
401100,4313,471,2033,72,001,06,401100
401200,2326,202,1284,58,001,06,401200
401300,3460,420,1676,114,001,06,401300
401400,4113,768,1416,142,001,06,401400
401500,1861,345,899,97,001,06,401500
403900,4108,383,2266,98,001,06,403900
404000,2548,248,1630,107,001,06,404000
404101,2724,215,1643,135,001,06,404101


In [468]:
print sum(basic10['Population (Estimate)'])
print sum(basic10['Total Households (Estimate)'])

36250.0
18423.0


In [469]:
agg(basic10)

Population (Estimate)             36250
Population (MOE)                1349.49
Total Households (Estimate)       18423
Total Households (MOE)         383.1971
county                                0
state                                 0
tract                                 0
Name: Aggregate, dtype: object

##### Modes of Transportation to Work

In [470]:
var_mode10 = vardict('commute','2010','ACS5')

In [471]:
var_mode10

{'B08301_001E': 'Workers (Estimate)',
 'B08301_001M': 'Workers (MOE)',
 'B08301_002E': 'Car (Estimate)',
 'B08301_002M': 'Car (MOE)',
 'B08301_010E': 'Transit (Estimate)',
 'B08301_010M': 'Transit (MOE)',
 'B08301_016E': 'Taxi (Estimate)',
 'B08301_016M': 'Taxi (MOE)',
 'B08301_018E': 'Bicycle (Estimate)',
 'B08301_018M': 'Bicycle (MOE)',
 'B08301_019E': 'Walked (Estimate)',
 'B08301_019M': 'Walked (MOE)'}

In [472]:
alameda_mode10 = acs5(var_mode10,CA,alameda,2012)
mode10 = tractfilter(alameda_mode10,tracts10)
print len(mode10)
mode10

11


,Workers (Estimate),Workers (MOE),Car (Estimate),Car (MOE),Transit (Estimate),Transit (MOE),Taxi (Estimate),Taxi (MOE),Bicycle (Estimate),Bicycle (MOE),Walked (Estimate),Walked (MOE),county,state,tract
tract,,,,,,,,,,,,,,,
401000,2529,331,1501,299,562,257,0,13,107,86,134,118,001,06,401000
401100,2417,310,1187,217,735,223,0,13,206,98,45,37,001,06,401100
401200,1443,236,887,208,275,106,0,13,153,71,53,44,001,06,401200
401300,1679,260,769,201,701,183,0,13,19,31,156,96,001,06,401300
401400,1471,422,825,270,321,145,0,13,110,79,165,146,001,06,401400
401500,905,192,485,142,255,85,0,13,42,32,15,19,001,06,401500
403900,2517,305,1699,279,420,202,0,13,105,85,138,104,001,06,403900
404000,1611,190,1116,161,282,85,0,13,54,43,59,41,001,06,404000
404101,1770,191,1100,149,385,136,0,13,93,78,126,86,001,06,404101


In [473]:
agg(mode10)

Workers (Estimate)       19465
Workers (MOE)         919.7179
Car (Estimate)           11387
Car (MOE)             729.4107
Transit (Estimate)        4461
Transit (MOE)         533.0216
Taxi (Estimate)              0
Taxi (MOE)            43.11612
Bicycle (Estimate)         962
Bicycle (MOE)         220.8778
Walked (Estimate)         1098
Walked (MOE)          281.5155
county                       0
state                        0
tract                        0
Name: Aggregate, dtype: object

In [474]:
mode10.to_csv('mode10.csv')
alameda_mode10.to_csv('alameda_mode10.csv')

##### Age distribution

A key problem here is that the recategorization function doesn't get margins of error. So I'll have to add that functionality in as well.

In [475]:
var_age10_E = vardict("age_est","2010","ACS5")
var_age10_M = vardict("age_moe","2010","ACS5")

In [476]:
alameda_age10E = acs5(var_age10_E,CA,alameda,2012)
age10E = tractfilter(alameda_age10E,tracts10)
age10E

,Total (Estimate),Male (Estimate),M Under 5 Years (Estimate),M 5 To 9 Years (Estimate),M 10 To 14 Years (Estimate),M 15 To 17 Years (Estimate),M 18 And 19 Years (Estimate),M 20 Years (Estimate),M 21 Years (Estimate),M 22 To 24 Years (Estimate),...,F 62 To 64 Years (Estimate),F 65 And 66 Years (Estimate),F 67 To 69 Years (Estimate),F 70 To 74 Years (Estimate),F 75 To 79 Years (Estimate),F 80 To 84 Years (Estimate),F 85 Years And Over (Estimate),county,state,tract
tract,,,,,,,,,,,,,,,,,,,,,
401000,4997,2284,112,206,121,18,16,0,43,77,...,53,42,17,20,60,117,94,001,06,401000
401100,4313,2109,119,181,42,0,25,0,0,140,...,109,24,20,8,35,7,13,001,06,401100
401200,2326,1006,54,30,12,13,0,0,0,0,...,34,40,47,16,0,10,56,001,06,401200
401300,3460,1879,27,51,71,9,14,0,80,69,...,25,22,20,61,31,23,68,001,06,401300
401400,4113,1877,158,186,65,107,0,57,34,155,...,119,0,35,56,17,28,34,001,06,401400
401500,1861,971,31,47,37,68,0,0,8,24,...,33,0,6,54,5,8,45,001,06,401500
403900,4108,1910,263,113,0,0,20,30,43,95,...,94,35,11,62,62,105,72,001,06,403900
404000,2548,1164,100,18,18,6,0,0,8,0,...,22,14,20,27,28,24,20,001,06,404000
404101,2724,1122,29,56,25,1,0,0,1,20,...,32,20,28,35,33,49,46,001,06,404101


In [477]:
alameda_age10M = acs5(var_age10_M,CA,alameda,2012)
age10M = tractfilter(alameda_age10M,tracts10)
len(age10M)

11

In [478]:
age10M.head()

,Total (MOE),Male (MOE),M Under 5 Years (MOE),M 5 To 9 Years (MOE),M 10 To 14 Years (MOE),M 15 To 17 Years (MOE),M 18 And 19 Years (MOE),M 20 Years (MOE),M 21 Years (MOE),M 22 To 24 Years (MOE),...,F 62 To 64 Years (MOE),F 65 And 66 Years (MOE),F 67 To 69 Years (MOE),F 70 To 74 Years (MOE),F 75 To 79 Years (MOE),F 80 To 84 Years (MOE),F 85 Years And Over (MOE),county,state,tract
tract,,,,,,,,,,,,,,,,,,,,,
401000,475,304,92,140,77,29,24,13,51,83,...,47,41,26,23,64,79,68,001,06,401000
401100,471,317,86,88,30,13,28,13,13,103,...,74,33,27,13,42,13,17,001,06,401100
401200,202,127,52,27,14,21,13,13,13,13,...,27,27,52,19,13,16,42,001,06,401200
401300,420,255,36,75,74,11,16,13,80,64,...,34,22,23,45,23,20,47,001,06,401300
401400,768,411,73,107,52,78,13,66,53,80,...,60,13,31,38,28,26,30,001,06,401400


In [479]:
agg(age10M)

Total (MOE)                   1349.49
Male (MOE)                   875.0371
M Under 5 Years (MOE)         280.125
M 5 To 9 Years (MOE)         248.0363
M 10 To 14 Years (MOE)       148.8657
M 15 To 17 Years (MOE)       124.6034
M 18 And 19 Years (MOE)      68.41053
M 20 Years (MOE)             86.01163
M 21 Years (MOE)             123.8144
M 22 To 24 Years (MOE)       223.8437
M 25 To 29 Years (MOE)       309.7499
M 30 To 34 Years (MOE)       304.4979
M 35 To 39 Years (MOE)       322.5756
M 40 To 44 Years (MOE)       226.2852
M 45 To 49 Years (MOE)       236.4001
M 50 To 54 Years (MOE)       247.4146
M 55 To 59 Years (MOE)         237.55
M 60 And 61 Years (MOE)      132.8194
M 62 To 64 Years (MOE)       157.3722
M 65 And 66 Years (MOE)      124.0726
M 67 To 69 Years (MOE)        80.4736
M 70 To 74 Years (MOE)       147.8141
M 75 To 79 Years (MOE)        116.383
M 80 To 84 Years (MOE)       88.87632
M 85 Years And Over (MOE)    107.6662
Female (MOE)                 892.9731
F Under 5 Ye

In [480]:
# quick checks on calculations

print sum(age10E['Male (Estimate)'])
print sum(age10E['Female (Estimate)'])
print sum(age10E['Male (Estimate)']) + sum(age10E['Female (Estimate)'])
print sum(basic10['Population (Estimate)'])

17209.0
19041.0
36250.0
72500.0


In [481]:
#upload recode sheet for age

agekey = pd.read_csv('agecategories.csv')
agekey.head()

,new,old
0,Under 18 (Estimate),M Under 5 Years (Estimate)
1,Under 18 (Estimate),M 5 To 9 Years (Estimate)
2,Under 18 (Estimate),M 10 To 14 Years (Estimate)
3,Under 18 (Estimate),M 15 To 17 Years (Estimate)
4,Under 18 (Estimate),F Under 5 Years (Estimate)


In [482]:
age10E_clean = collapse(age10E,agekey)

In [483]:
age10M_clean = collapseMOE(age10M,agekey)

In [484]:
age10_clean = concatenate_dataframes(age10E_clean,age10M_clean)

In [485]:
agg(age10_clean)

Under 18 (Estimate)        5414.000000
18 to 34 (Estimate)       11426.000000
35 to 49 (Estimate)        4500.000000
50 to 64 (Estimate)        9769.000000
65 and over (Estimate)     5141.000000
Under 18 (MOE)              579.907751
18 to 34 (MOE)              761.357341
35 to 49 (MOE)              459.505169
50 to 64 (MOE)              674.137968
65 and over (MOE)           495.058582
Name: Aggregate, dtype: float64

##### Race

In [486]:
var_racehisp10 = vardict("race_hisp","2010","ACS5")

In [487]:
alameda_racehisp10 = acs5(var_racehisp10,CA,alameda,2012)
racehisp10 = tractfilter(alameda_racehisp10,tracts10)
racehisp10

,Total (Estimate),Total (MOE),Not Hispanic Or Latino (Estimate),Not Hispanic Or Latino (MOE),White Alone (Estimate),White Alone (MOE),Black Or African American Alone (Estimate),Black Or African American Alone (MOE),American Indian And Alaska Native Alone (Estimate),American Indian And Alaska Native Alone (MOE),...,Native Hawaiian And Other Pacific Islander Alone (MOE),Some Other Race Alone (Estimate),Some Other Race Alone (MOE),Two Or More Races (Estimate),Two Or More Races (MOE),Hispanic Or Latino (Estimate),Hispanic Or Latino (MOE),county,state,tract
tract,,,,,,,,,,,,,,,,,,,,,
401000,4997,475,4518,525,1115,261,2747,396,0,13,...,125,0,13,251,175,479,188,001,06,401000
401100,4313,471,3535,452,2017,295,798,264,26,32,...,49,0,13,163,87,778,282,001,06,401100
401200,2326,202,2202,213,1419,232,435,178,0,13,...,52,0,13,56,48,124,69,001,06,401200
401300,3460,420,2739,354,714,189,1237,282,18,22,...,2,0,13,93,62,721,261,001,06,401300
401400,4113,768,3294,485,533,316,2072,361,74,75,...,13,32,38,68,79,819,594,001,06,401400
401500,1861,345,1730,377,586,226,942,250,0,13,...,13,0,13,64,43,131,99,001,06,401500
403900,4108,383,3803,396,2074,302,1030,332,0,13,...,13,30,38,231,137,305,127,001,06,403900
404000,2548,248,2321,253,1489,233,343,178,0,13,...,13,16,26,160,74,227,99,001,06,404000
404101,2724,215,2531,303,1918,259,33,38,7,12,...,13,7,12,188,106,193,164,001,06,404101


In [488]:
agg(racehisp10)

Total (Estimate)                                                  36250
Total (MOE)                                                     1349.49
Not Hispanic Or Latino (Estimate)                                 31996
Not Hispanic Or Latino (MOE)                                    1241.23
White Alone (Estimate)                                            14286
White Alone (MOE)                                              864.8063
Black Or African American Alone (Estimate)                        10966
Black Or African American Alone (MOE)                           892.743
American Indian And Alaska Native Alone (Estimate)                  125
American Indian And Alaska Native Alone (MOE)                  91.97826
Asian Alone (Estimate)                                             4770
Asian Alone (MOE)                                               673.629
Native Hawaiian And Other Pacific Islander Alone (Estimate)         163
Native Hawaiian And Other Pacific Islander Alone (MOE)         1

##### Hispanic Origin

In [489]:
var_hisp10 = vardict("hispanic","2010","ACS5")

In [490]:
alameda_hisp10 = acs5(var_hisp10,CA,alameda,2012)
hisp10 = tractfilter(alameda_hisp10,tracts10)
len(hisp10)

11

In [491]:
agg(hisp10)

Total (Estimate)                    36250
Total (MOE)                       1349.49
Hispanic Or Latino (Estimate)        4254
Hispanic Or Latino (MOE)         792.5011
county                                  0
state                                   0
tract                                   0
Name: Aggregate, dtype: object

##### Poverty

In [492]:
var_pov10 = vardict("poverty","2010","ACS5")

In [493]:
alameda_pov10 = acs5(var_pov10,CA,alameda,2012)
pov10 = tractfilter(alameda_pov10,tracts10)
len(pov10)

11

In [494]:
agg(pov10)

Total (Estimate)                                                  35632
Total (MOE)                                                    1336.997
Income In The Past 12 Months Below Poverty Level (Estimate)        6864
Income In The Past 12 Months Below Poverty Level (MOE)         794.0497
county                                                                0
state                                                                 0
tract                                                                 0
Name: Aggregate, dtype: object

##### Household Income

In [495]:
var_hhincome10 = vardict("hhincome","2010","ACS5")

In [496]:
alameda_hhincome10 = acs5(var_hhincome10,CA,alameda,2012)
hhincome10 = tractfilter(alameda_hhincome10,tracts10)
len(hhincome10)

11

In [497]:
agg(hhincome10)

Total (Estimate)                      18423
Total (MOE)                        383.1971
Less Than $10,000 (Estimate)           1469
Less Than $10,000 (MOE)             250.108
$10,000 To $14,999 (Estimate)          1409
$10,000 To $14,999 (MOE)           264.3426
$15,000 To $19,999 (Estimate)          1043
$15,000 To $19,999 (MOE)           220.0091
$20,000 To $24,999 (Estimate)          1205
$20,000 To $24,999 (MOE)           257.5092
$25,000 To $29,999 (Estimate)           765
$25,000 To $29,999 (MOE)             196.84
$30,000 To $34,999 (Estimate)           990
$30,000 To $34,999 (MOE)           221.2216
$35,000 To $39,999 (Estimate)           707
$35,000 To $39,999 (MOE)           197.4234
$40,000 To $44,999 (Estimate)           694
$40,000 To $44,999 (MOE)            174.462
$45,000 To $49,999 (Estimate)           957
$45,000 To $49,999 (MOE)           241.5326
$50,000 To $59,999 (Estimate)          1818
$50,000 To $59,999 (MOE)           328.2408
$60,000 To $74,999 (Estimate)   

##### Education

In [498]:
var_education10 = vardict("education","2010","ACS5")

In [499]:
alameda_education10 = acs5(var_education10,CA,alameda,2012)
education10 = tractfilter(alameda_education10,tracts10)
len(education10)

11

In [500]:
agg(education10)

Total (Estimate)                                          28248
Total (MOE)                                            912.4895
No Schooling Completed (Estimate)                           184
No Schooling Completed (MOE)                           97.04123
Nursery School (Estimate)                                    10
Nursery School (MOE)                                   46.16276
Kindergarten (Estimate)                                      33
Kindergarten (MOE)                                     58.77074
1st Grade (Estimate)                                         10
1st Grade (MOE)                                        46.16276
2nd Grade (Estimate)                                         78
2nd Grade (MOE)                                        60.32412
3rd Grade (Estimate)                                         33
3rd Grade (MOE)                                        42.43819
4th Grade (Estimate)                                         49
4th Grade (MOE)                         

##### Tenure

In [501]:
var_tenure10 = vardict("tenure","2010","ACS5")

In [502]:
alameda_tenure10 = acs5(var_tenure10,CA,alameda,2012)
tenure10 = tractfilter(alameda_tenure10,tracts10)
len(tenure10)

11

In [503]:
agg(tenure10)

Total (Estimate)                 18423
Total (MOE)                   383.1971
Owner Occupied (Estimate)         5035
Owner Occupied (MOE)          388.9871
Renter Occupied (Estimate)       13388
Renter Occupied (MOE)         487.8391
county                               0
state                                0
tract                                0
Name: Aggregate, dtype: object

##### Vehicles

In [504]:
var_veh10E = vardict("vehicles_est","2010","ACS5")
var_veh10M = vardict("vehicles_moe","2010","ACS5")

In [505]:
vehkey10 = pd.read_csv('vehcategories.csv')
vehkey00 = pd.read_csv('vehcategories00.csv')

In [506]:
alameda_vehicles10E = acs5(var_veh10E,CA,alameda,2012)
vehicles10E = tractfilter(alameda_vehicles10E,tracts10)
len(vehicles10E)

11

In [507]:
alameda_vehicles10M = acs5(var_veh10M,CA,alameda,2012)
vehicles10M = tractfilter(alameda_vehicles10M,tracts10)
len(vehicles10M)

11

In [508]:
vehicles10E_clean = collapse(vehicles10E,vehkey10)
vehicles10M_clean = collapse(vehicles10M,vehkey10)

In [509]:
vehicles10 = concatenate_dataframes(vehicles10E_clean,vehicles10M_clean)

In [510]:
vehicles10

,Total (Estimate),No Vehicle (Estimate),One Vehicle (Estimate),Two or More Vehicles (Estimate),Total (MOE),No Vehicle (MOE),One Vehicle (MOE),Two or More Vehicles (MOE)
tract,,,,,,,,
401000,2288,527,1093,668,140,230,335,441
401100,2033,560,1066,407,72,190,238,289
401200,1284,225,636,423,58,110,189,280
401300,1676,751,680,245,114,171,208,268
401400,1416,484,633,299,142,131,216,256
401500,899,304,347,248,97,118,131,209
403900,2266,196,1498,572,98,139,323,367
404000,1630,258,965,407,107,109,228,231
404101,1643,302,900,441,135,147,246,239


In [511]:
vehkey10

,new,old
0,Total (Estimate),Total (Estimate)
1,No Vehicle (Estimate),Owner-No Vehicle Available (Estimate)
2,One Vehicle (Estimate),Owner-1 Vehicle Available (Estimate)
3,Two or More Vehicles (Estimate),Owner-2 Vehicles Available (Estimate)
4,Two or More Vehicles (Estimate),Owner-3 Vehicles Available (Estimate)
5,Two or More Vehicles (Estimate),Owner-4 Vehicles Available (Estimate)
6,Two or More Vehicles (Estimate),Owner-5 Or More Vehicles Available (Estimate)
7,No Vehicle (Estimate),Renter-No Vehicle Available (Estimate)
8,One Vehicle (Estimate),Renter-1 Vehicle Available (Estimate)
9,Two or More Vehicles (Estimate),Renter-2 Vehicles Available (Estimate)


In [512]:
agg(vehicles10)

Total (Estimate)                   18423.000000
No Vehicle (Estimate)               4292.000000
One Vehicle (Estimate)              9682.000000
Two or More Vehicles (Estimate)     4449.000000
Total (MOE)                          383.197077
No Vehicle (MOE)                     520.022115
One Vehicle (MOE)                    801.526668
Two or More Vehicles (MOE)           965.727705
Name: Aggregate, dtype: float64

## Decennial Census 2000 Descriptives

##### Basic Descriptives

In [513]:
var_basic00 = vardict('basic','2000','SF1')

In [514]:
var_basic00

{'P001001': 'Population', 'P015001': 'Households'}

In [515]:
alameda_basic00 = sf1(var_basic00,CA,alameda,2000)
basic00 = tractfilter(alameda_basic00,tracts00)
len(basic00)

9

In [516]:
basic00

,Population,Households,county,state,tract
tract,,,,,
401000,5709,2299,1,6,401000
401100,4007,1941,1,6,401100
401200,2432,1200,1,6,401200
401300,2810,1345,1,6,401300
401400,4765,1488,1,6,401400
401500,2413,911,1,6,401500
403900,3794,2178,1,6,403900
404000,2885,1781,1,6,404000
404100,5337,3356,1,6,404100


In [517]:
agg(basic00)

Population    34152
Households    16499
county            0
state             0
tract             0
Name: Aggregate, dtype: object

##### Means of Transportation to Work

In [518]:
var_commute00 = vardict('commute','2000','SF3')

In [519]:
alameda_commute00 = sf3(var_commute00,CA,alameda,2000)
commute00 = tractfilter(alameda_commute00,tracts00)
len(commute00)

9

In [520]:
agg(commute00)

Total                    16721
Car, truck, or van       10377
Public transportation     4115
Bicycle                    528
Walked                     727
county                       0
state                        0
tract                        0
Name: Aggregate, dtype: object

##### Age

In [521]:
var_age00_1 = vardict('age1','2000','SF3')
var_age00_2 = vardict('age2','2000','SF3')

In [522]:
alameda_age00_1 = sf3(var_age00_1,CA,alameda,2000)
alameda_age00_2 = sf3(var_age00_2,CA,alameda,2000)

In [523]:
age00_1 = tractfilter(alameda_age00_1,tracts00)
age00_2 = tractfilter(alameda_age00_2,tracts00)

In [524]:
age00 = concatenate_dataframes(age00_1,age00_2)
age00.head()

,Total:,Male:,Under 1 year M,1 year M,2 years M,3 years M,4 years M,5 years M,6 years M,7 years M,...,62 to 64 years F,65 and 66 years F,67 to 69 years F,70 to 74 years F,75 to 79 years F,80 to 84 years F,85 years and over F,county,state,tract
tract,,,,,,,,,,,,,,,,,,,,,
401000,5709,2617,45,0,15,11,54,21,38,67,...,49,13,54,118,63,109,36,1,6,401000
401100,4007,1914,5,16,23,15,22,22,20,19,...,11,0,26,36,16,7,23,1,6,401100
401200,2322,1068,14,23,0,0,17,0,9,0,...,32,0,11,31,19,29,18,1,6,401200
401300,2920,1472,13,3,24,29,12,0,11,21,...,21,49,30,28,12,71,98,1,6,401300
401400,4765,2383,51,21,47,77,50,50,35,92,...,65,34,34,34,44,47,21,1,6,401400


In [525]:
agekeys00 = pd.read_csv("agecategories00.csv")
agekeys00.head()

,new,old
0,Total,Total:
1,Under 18,Under 1 year M
2,Under 18,1 year M
3,Under 18,2 years M
4,Under 18,3 years M


In [526]:
age00_clean = collapse(age00,agekeys00)

In [527]:
agg(age00_clean)

Total          34317
Under 18        6474
18 to 34       10775
35 to 49        8676
50 to 64        4468
65 and over     3924
Name: Aggregate, dtype: float64

##### Race

In [528]:
var_racehisp00 = vardict('race_hisp','2000','SF3')

In [529]:
alameda_racehisp00 = sf3(var_racehisp00,CA,alameda,2000)
racehisp00 = tractfilter(alameda_racehisp00,tracts00)

In [530]:
agg(racehisp00)

Total                                               34317
Not Hispanic or Latino                              31109
White alone                                         11848
Black or African American alone                     13796
American Indian and Alaska Native alone               159
Asian alone                                          3140
Native Hawaiian and Other Pacific Islander alone       76
Some other race alone                                 158
Two or more races                                    1932
Hispanic or Latino                                   3208
county                                                  0
state                                                   0
tract                                                   0
Name: Aggregate, dtype: object

##### Hispanic

In [531]:
var_hisp00 = vardict('hispanic','2000','SF3')

In [532]:
alameda_hisp00 = sf3(var_hisp00,CA,alameda,2000)
hisp00 = tractfilter(alameda_hisp00,tracts00)

In [533]:
agg(hisp00)

Total                 34317
Hispanic or Latino     3208
county                    0
state                     0
tract                     0
Name: Aggregate, dtype: object

##### Poverty

In [534]:
var_pov00 = vardict('poverty','2000','SF3')

In [535]:
alameda_pov00 = sf3(var_pov00,CA,alameda,2000)
pov00 = tractfilter(alameda_pov00,tracts00)

In [536]:
agg(pov00)

Total                                 33911
Income in 1999 below poverty level     8316
county                                    0
state                                     0
tract                                     0
Name: Aggregate, dtype: object

##### Income

Income variable used from ACS is not comparable to 2000. If you want to look at this over time, need to find another comparable variable.

##### Education

Same as above.

##### Tenure

In [537]:
var_tenure00 = vardict('tenure','2000','SF3')

In [538]:
alameda_tenure00 = sf3(var_tenure00,CA,alameda,2000)
tenure00 = tractfilter(alameda_tenure00,tracts00)

In [539]:
agg(tenure00)

Total              16509
Owner occupied      3891
Renter occupied    12618
county                 0
state                  0
tract                  0
Name: Aggregate, dtype: object

##### Vehicles

In [540]:
var_veh00 = vardict("vehicles","2000","SF3")

In [541]:
alameda_vehicles00 = sf3(var_veh00,CA,alameda,2000)
vehicles00 = tractfilter(alameda_vehicles00,tracts00)
len(vehicles00)

9

In [542]:
vehicles00

,Total,Owner occupied,Owner-No vehicle available,Owner-1 vehicle available,Owner-2 vehicles available,Owner-3 vehicles available,Owner-4 vehicles available,Owner-5 or more vehicles available,Renter occupied,Renter-No vehicle available,Renter-1 vehicle available,Renter-2 vehicles available,Renter-3 vehicles available,Renter-4 vehicles available,Renter-5 or more vehicles available,county,state,tract
tract,,,,,,,,,,,,,,,,,,
401000,2299,738,122,368,167,61,20,0,1561,535,755,201,27,36,7,1,6,401000
401100,1941,354,50,151,108,39,0,6,1587,443,844,268,20,12,0,1,6,401100
401200,1183,389,27,177,110,75,0,0,794,107,503,134,37,0,13,1,6,401200
401300,1362,60,0,24,22,14,0,0,1302,638,531,108,18,0,7,1,6,401300
401400,1488,262,32,78,114,38,0,0,1226,480,608,113,25,0,0,1,6,401400
401500,921,202,27,94,47,26,0,8,719,345,243,95,31,5,0,1,6,401500
403900,2178,643,62,373,135,64,9,0,1535,213,996,303,18,5,0,1,6,403900
404000,1781,417,20,251,115,31,0,0,1364,182,869,299,14,0,0,1,6,404000
404100,3356,826,68,392,302,44,20,0,2530,723,1414,340,37,9,7,1,6,404100


In [543]:
vehkey00

,new,old
0,Total (Estimate),Total
1,No Vehicle (Estimate),Owner-No vehicle available
2,One Vehicle (Estimate),Owner-1 vehicle available
3,Two or More Vehicles (Estimate),Owner-2 vehicles available
4,Two or More Vehicles (Estimate),Owner-3 vehicles available
5,Two or More Vehicles (Estimate),Owner-4 vehicles available
6,Two or More Vehicles (Estimate),Owner-5 or more vehicles available
7,No Vehicle (Estimate),Renter-No vehicle available
8,One Vehicle (Estimate),Renter-1 vehicle available
9,Two or More Vehicles (Estimate),Renter-2 vehicles available


In [544]:
vehicles00_clean = collapse(vehicles00,vehkey00)

In [545]:
vehicles00_clean

,Total (Estimate),No Vehicle (Estimate),One Vehicle (Estimate),Two or More Vehicles (Estimate)
tract,,,,
401000,2299,657,1123,519
401100,1941,493,995,453
401200,1183,134,680,369
401300,1362,638,555,169
401400,1488,512,686,290
401500,921,372,337,212
403900,2178,275,1369,534
404000,1781,202,1120,459
404100,3356,791,1806,759


In [546]:
agg(vehicles00_clean)

Total (Estimate)                   16509
No Vehicle (Estimate)               4074
One Vehicle (Estimate)              8671
Two or More Vehicles (Estimate)     3764
Name: Aggregate, dtype: float64

# Alameda County Descriptives

##### 2008-2010 ACS

In [547]:
agg(alameda_basic10)

Population (Estimate)           1515136
Population (MOE)               8006.224
Total Households (Estimate)      539179
Total Households (MOE)         1804.844
county                                0
state                                 0
tract                                 0
Name: Aggregate, dtype: object

In [548]:
alameda_age10E_clean = collapse(alameda_age10E,agekey)
alameda_age10M_clean = collapseMOE(alameda_age10M,agekey)

In [549]:
alameda_age10_clean = concatenate_dataframes(alameda_age10E_clean, alameda_age10M_clean)

In [550]:
agg(alameda_age10_clean)

Under 18 (Estimate)       340863.000000
18 to 34 (Estimate)       379251.000000
35 to 49 (Estimate)       169705.000000
50 to 64 (Estimate)       413842.000000
65 and over (Estimate)    211475.000000
Under 18 (MOE)              4231.550661
18 to 34 (MOE)              4699.308885
35 to 49 (MOE)              2798.284474
50 to 64 (MOE)              4112.038910
65 and over (MOE)           2866.296042
Name: Aggregate, dtype: float64

In [551]:
agg(alameda_racehisp10)

Total (Estimate)                                                1515136
Total (MOE)                                                    8006.224
Not Hispanic Or Latino (Estimate)                               1175695
Not Hispanic Or Latino (MOE)                                   7171.967
White Alone (Estimate)                                           516580
White Alone (MOE)                                               5082.89
Black Or African American Alone (Estimate)                       181711
Black Or African American Alone (MOE)                          4279.047
American Indian And Alaska Native Alone (Estimate)                 4318
American Indian And Alaska Native Alone (MOE)                  596.8434
Asian Alone (Estimate)                                           397475
Asian Alone (MOE)                                              5498.328
Native Hawaiian And Other Pacific Islander Alone (Estimate)       12442
Native Hawaiian And Other Pacific Islander Alone (MOE)         1

In [552]:
agg(alameda_pov10)

Total (Estimate)                                                1487888
Total (MOE)                                                    7903.535
Income In The Past 12 Months Below Poverty Level (Estimate)      178897
Income In The Past 12 Months Below Poverty Level (MOE)         5093.676
county                                                                0
state                                                                 0
tract                                                                 0
Name: Aggregate, dtype: object

In [553]:
agg(alameda_education10)

Total (Estimate)                                        1025190
Total (MOE)                                            5269.772
No Schooling Completed (Estimate)                         21320
No Schooling Completed (MOE)                           1233.279
Nursery School (Estimate)                                   144
Nursery School (MOE)                                   280.2749
Kindergarten (Estimate)                                     264
Kindergarten (MOE)                                     289.3475
1st Grade (Estimate)                                       1134
1st Grade (MOE)                                        381.6648
2nd Grade (Estimate)                                       2396
2nd Grade (MOE)                                        450.2066
3rd Grade (Estimate)                                       4280
3rd Grade (MOE)                                         558.541
4th Grade (Estimate)                                       3718
4th Grade (MOE)                         

In [554]:
agg(alameda_hhincome10)

Total (Estimate)                     539179
Total (MOE)                        1804.844
Less Than $10,000 (Estimate)          27962
Less Than $10,000 (MOE)            1143.973
$10,000 To $14,999 (Estimate)         26522
$10,000 To $14,999 (MOE)           1093.781
$15,000 To $19,999 (Estimate)         21832
$15,000 To $19,999 (MOE)            1003.02
$20,000 To $24,999 (Estimate)         22214
$20,000 To $24,999 (MOE)            1027.74
$25,000 To $29,999 (Estimate)         20072
$25,000 To $29,999 (MOE)           970.7945
$30,000 To $34,999 (Estimate)         18978
$30,000 To $34,999 (MOE)           953.3452
$35,000 To $39,999 (Estimate)         18817
$35,000 To $39,999 (MOE)           949.0622
$40,000 To $44,999 (Estimate)         19120
$40,000 To $44,999 (MOE)           960.9729
$45,000 To $49,999 (Estimate)         18315
$45,000 To $49,999 (MOE)           942.9793
$50,000 To $59,999 (Estimate)         37308
$50,000 To $59,999 (MOE)           1318.479
$60,000 To $74,999 (Estimate)   

In [555]:
agg(alameda_tenure10)

Total (Estimate)                539179
Total (MOE)                   1804.844
Owner Occupied (Estimate)       289758
Owner Occupied (MOE)          2104.572
Renter Occupied (Estimate)      249421
Renter Occupied (MOE)         2284.463
county                               0
state                                0
tract                                0
Name: Aggregate, dtype: object

In [556]:
agg(alameda_mode10)

Workers (Estimate)      693960
Workers (MOE)         5055.298
Car (Estimate)          527826
Car (MOE)             4553.893
Transit (Estimate)       82417
Transit (MOE)         2178.467
Taxi (Estimate)            301
Taxi (MOE)            295.8209
Bicycle (Estimate)       11945
Bicycle (MOE)          821.624
Walked (Estimate)        26202
Walked (MOE)          1308.204
county                       0
state                        0
tract                        0
Name: Aggregate, dtype: object

In [566]:
alameda_vehicles10E_clean = collapse(alameda_vehicles10E,vehkey10)
alameda_vehicles10M_clean = collapseMOE(alameda_vehicles10M,vehkey10)

In [570]:
alameda_vehicles10_clean = concatenate_dataframes(alameda_vehicles10E_clean,alameda_vehicles10M_clean)

In [571]:
agg(alameda_vehicles10_clean)

Total (Estimate)                   539179.000000
No Vehicle (Estimate)               56414.000000
One Vehicle (Estimate)             182022.000000
Two or More Vehicles (Estimate)    300743.000000
Total (MOE)                          1804.844315
No Vehicle (MOE)                     1509.853966
One Vehicle (MOE)                    2559.597429
Two or More Vehicles (MOE)           3305.458365
Name: Aggregate, dtype: float64

##### 2000 Census SF3

In [557]:
agg(alameda_basic00)

Population    1443741
Households     523366
county              0
state               0
tract               0
Name: Aggregate, dtype: object

In [558]:
alameda_age00 = concatenate_dataframes(alameda_age00_1,alameda_age00_2)

In [559]:
alameda_age00_clean = collapse(alameda_age00,agekeys00)

In [560]:
agg(alameda_age00_clean)

Total          1443741
Under 18        353460
18 to 34        374751
35 to 49        364049
50 to 64        203818
65 and over     147663
Name: Aggregate, dtype: float64

In [561]:
agg(alameda_racehisp00)

Total                                               1443741
Not Hispanic or Latino                              1169854
White alone                                          589243
Black or African American alone                      208559
American Indian and Alaska Native alone                5548
Asian alone                                          291493
Native Hawaiian and Other Pacific Islander alone       8729
Some other race alone                                  4319
Two or more races                                     61963
Hispanic or Latino                                   273887
county                                                    0
state                                                     0
tract                                                     0
Name: Aggregate, dtype: object

In [562]:
agg(alameda_pov00)

Total                                 1419998
Income in 1999 below poverty level     156804
county                                      0
state                                       0
tract                                       0
Name: Aggregate, dtype: object

In [563]:
agg(alameda_tenure00)

Total              523366
Owner occupied     286306
Renter occupied    237060
county                  0
state                   0
tract                   0
Name: Aggregate, dtype: object

In [564]:
agg(alameda_commute00)

Total                    678910
Car, truck, or van       544148
Public transportation     72174
Bicycle                    8385
Walked                    21919
county                        0
state                         0
tract                         0
Name: Aggregate, dtype: object

In [572]:
alameda_vehicles00_clean = collapse(alameda_vehicles00,vehkey00)

In [573]:
agg(alameda_vehicles00_clean)

Total (Estimate)                   523366
No Vehicle (Estimate)               57287
One Vehicle (Estimate)             182586
Two or More Vehicles (Estimate)    283493
Name: Aggregate, dtype: float64